In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "data_prep", ENTITY_NAME: "coco_test_dataset"}, # dataset for detector testing from data_prep step
      { STEP_NAME: "model_train", ENTITY_NAME: "obj_detect_inference_files"}, # detector files from train step
      { STEP_NAME: "model_pack", ENTITY_NAME: "bento_service"} # bentoservice file from pack step
    ],
    
    tmp_entities =
    [
        { ENTITY_NAME: "coco_test_dataset" },  # ground-true test dataset files for use in next substep
        { ENTITY_NAME: "obj_detect_inference_files" }, # temporary detector files from train step
        { ENTITY_NAME: "bentoservice_artifacts" } # stored BentoService
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
import os.path as osp
from pathlib import Path
import json
import torch
from sinara.bentoml.utils import compute_md5

In [ ]:
# run spark
from sinara.spark import SinaraSpark
from sinara.archive import SinaraArchive

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading obj_detect_inference_files the model_train step 
(weights, configs, reference test image)

In [ ]:
inputs = substep.inputs(step_name = "model_train")
tmp_entities = substep.tmp_entities()

archive = SinaraArchive(spark)
archive.unpack_files_from_store_to_tmp(store_path=inputs.obj_detect_inference_files, tmp_entity_dir=tmp_entities.obj_detect_inference_files)

### Loading binary bento_service the model_pack step and extracting artifacts

In [ ]:
from sinara.bentoml import extract_artifacts_from_bentoservice

inputs = substep.inputs(step_name = "model_pack")

artifacts_folder = extract_artifacts_from_bentoservice(inputs.bento_service, tmp_entities.bentoservice_artifacts)
print(f'Aftifacts extracted to {artifacts_folder}')

### Compute hash of the reference test image

In [ ]:
reference_test_image_path = Path(tmp_entities.obj_detect_inference_files) / "test.jpg"
assert reference_test_image_path.exists()
reference_test_image_hash = compute_md5(reference_test_image_path)

print(f'Reference image hash: {reference_test_image_hash}')

### Compute hash of the test image extracted from bentoservice

In [ ]:
test_image_path = Path(artifacts_folder) / 'artifacts' / 'test_image.jpg'
assert test_image_path.exists()
test_image_hash = compute_md5(test_image_path)
print(f'Test image hash: {test_image_hash}')

### Check if the test image has been extracted correctly

In [ ]:
assert reference_test_image_hash == test_image_hash

### Check inference test image by model from artifacts of bento_service
### and inference test image by model from model_train

In [ ]:
from mmdet.apis import init_detector, inference_detector

reference_test_image_path = Path(tmp_entities.obj_detect_inference_files) / "test.jpg"

# build detector from a config file and a checkpoint file from bento_archive
detector_from_bento_archive = init_detector(osp.join(artifacts_folder, 'artifacts/mmengine_cfg.py'), 
                                        osp.join(artifacts_folder, 'artifacts/weigths_pth.pth'), 
                                        device='cpu')
# detector_from_bento_archive on test image
detection_result_from_bentoservice = inference_detector(detector_from_bento_archive, [reference_test_image_path])


# build the detector from a config file and a checkpoint file from model_train
detector_from_inference_files = init_detector(osp.join(tmp_entities.obj_detect_inference_files, 'last_cfg.py'), 
                                           osp.join(tmp_entities.obj_detect_inference_files, 'latest_checkpoint.pth'), 
                                           device='cpu')
# run detector_from_inference_files on test image
detection_result_from_model_train = inference_detector(detector_from_inference_files, [reference_test_image_path])    

detection_result_from_bentoservice = detection_result_from_bentoservice[0]
detection_result_from_model_train = detection_result_from_model_train[0]

#### Compare results of detection

In [ ]:
from utils.bb_intersection_over_union import bb_intersection_over_union

_success = True

try:    
    
    for bbox_model_train, label_model_train in zip(detection_result_from_model_train.pred_instances.bboxes,
                                                   detection_result_from_model_train.pred_instances.labels):
        ious = []
        for bbox_bento, label_bento in zip(detection_result_from_bentoservice.pred_instances.bboxes,
                                           detection_result_from_bentoservice.pred_instances.labels):
            if label_model_train != label_bento:  # compare by object class
                continue
            ious.append(bb_intersection_over_union(bbox_model_train, bbox_bento))
            
        id_bbox_iou = torch.tensor(ious).argmax()  
        if ious[id_bbox_iou] < 0.98:  # no matches by iou     
            _success = False
            
except Exception as e:
    print(f"error: {e.__str__()}")
    _success = False

In [ ]:
assert _success

In [ ]:
# Stop spark
SinaraSpark.stop_session()